In [ ]:
# Making some essential imports as Usual
import os
import sys
import numpy as np
import re
import cv2 as op
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from os import listdir
import pandas as pd
from keras.layers import Dense, Dropout, Input, add, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose,Activation, Concatenate
from tensorflow import keras
from time import time
from tqdm import tqdm
from keras import backend as K

# plt.style.use('seaborn')
tf.__version__

In [ ]:
!pip install albumentations > /dev/null
!pip install -U efficientnet==0.0.4
import numpy as np
import pandas as pd
import gc
import keras

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split,StratifiedKFold

from skimage.transform import resize
import tensorflow as tf
import keras.backend as K
from keras.losses import binary_crossentropy

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import  ModelCheckpoint
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file

from keras import backend as K
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.losses import binary_crossentropy
import keras.callbacks as callbacks
from keras.callbacks import Callback
from keras.applications.xception import Xception
from keras.layers import multiply


In [ ]:
from keras.utils.data_utils import get_file

In [ ]:
%env SM_FRAMEWORK = tf.keras
!pip install segmentation_models
!pip install  albumentations

In [ ]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

In [ ]:
img = plt.imread('../input/massachusetts-roads-dataset/tiff/train/10228675_15.tiff')
print(img.shape)

plt.imshow(img)

In [ ]:
def build_data(PATH):
    data = []
    filenames = sorted(listdir(PATH))
    for name in filenames:
        data.append(PATH + '/' + name)
    return data

    
TRAIN_PATH = '../input/massachusetts-roads-dataset/tiff/train'
LABEL_PATH = '../input/massachusetts-roads-dataset/tiff/train_labels'

sat_image = build_data(TRAIN_PATH)
mask_image = build_data(LABEL_PATH)

VAL_TRAIN_PATH = '../input/massachusetts-roads-dataset/tiff/val'
VAL_LABEL_PATH = '../input/massachusetts-roads-dataset/tiff/val_labels'

VAL_sat_image = build_data(VAL_TRAIN_PATH)
VAL_mask_image = build_data(VAL_LABEL_PATH)

In [ ]:
TEST_PATH = '../input/massachusetts-roads-dataset/tiff/test'
TEST_LABEL_PATH = '../input/massachusetts-roads-dataset/tiff/test_labels'

TEST_sat_image = build_data(TEST_PATH)
TEST_mask_image = build_data(TEST_LABEL_PATH)

In [ ]:
len(sat_image), len(mask_image), len(VAL_sat_image), len(VAL_mask_image ),len(TEST_sat_image), len(TEST_mask_image )

In [ ]:
df = pd.DataFrame({
    'Image' : sat_image,
    'Mask' : mask_image
})

In [ ]:
val_df = pd.DataFrame({
    'Val_Image' : VAL_sat_image,
    'Val_Mask' : VAL_mask_image
})

In [ ]:
test_df = pd.DataFrame({
    'Test_Image' : TEST_sat_image,
    'Test_Mask' : TEST_mask_image
})

In [ ]:
import albumentations as A
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Blur(blur_limit = 5, p = 0.85), 
#     A.RandomCrop(height = 512, width = 512, p = 1)
])

In [ ]:
IMG_SIZE = (256, 256)

def modify_mask(mask):
    mask = np.expand_dims(mask, axis = 2)
    t_mask = np.zeros(mask.shape)
    np.place(t_mask[:, :, 0], mask[:, :, 0] >=100, 1)
    return t_mask

def map_function(img, mask):
    img, mask = plt.imread(img.decode()), plt.imread(mask.decode())
    img = op.resize(img, IMG_SIZE)
    mask = modify_mask(op.resize(mask, IMG_SIZE))
    
    img = img/255.0
    transformed = transform(image=img, mask=mask)
    img = transformed['image']
    mask = transformed['mask']

    #mask = modify_mask(mask)
    
    return img.astype(np.float64), mask.astype(np.float64)

In [ ]:
BATCH_SIZE = 16

def create_dataset(data):
    dataset = tf.data.Dataset.from_tensor_slices((data['Image'], data['Mask']))
    dataset = dataset.shuffle(1000)
    dataset = dataset.map(lambda img, mask : tf.numpy_function(
                    map_function, [img, mask], [tf.float64, tf.float64]),
                    num_parallel_calls = tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE)

    dataset = dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
BATCH_SIZE = 16

def create_dataset1(data):
    dataset = tf.data.Dataset.from_tensor_slices((data['Val_Image'], data['Val_Mask']))
    dataset = dataset.shuffle(1000)
    dataset = dataset.map(lambda img, mask : tf.numpy_function(
                    map_function, [img, mask], [tf.float64, tf.float64]),
                    num_parallel_calls = tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE)

    dataset = dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
BATCH_SIZE = 16

def create_dataset2(data):
    dataset = tf.data.Dataset.from_tensor_slices((data['Test_Image'], data['Test_Mask']))
    dataset = dataset.shuffle(1000)
    dataset = dataset.map(lambda img, mask : tf.numpy_function(
                    map_function, [img, mask], [tf.float64, tf.float64]),
                    num_parallel_calls = tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE)

    dataset = dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
train_dataset = create_dataset(df)
img, mask = next(iter(train_dataset))
img.shape, mask.shape

In [ ]:
val_dataset = create_dataset1(val_df)
val_img, val_mask = next(iter(val_dataset))
val_img.shape, val_mask.shape

In [ ]:
test_dataset = create_dataset2(test_df)
test_img, test_mask = next(iter(test_dataset))
test_img.shape, test_mask.shape